In [ ]:
import pandas as pd

# Load the Excel file
file_path = r"file_path"

# Step 1: Load the data
sheet1 = pd.read_excel(file_path, sheet_name='Somanya Farmers Geoboundaries')
sheet2 = pd.read_excel(file_path, sheet_name='Names')

# Step 2: Define a function to process the "full_name" column
def process_full_name(row, name_reference):
    full_name = row['full_name']
    farm_number = None

    # Check if "Farm X" exists and extract it
    if "Farm" in full_name:
        parts = full_name.split("Farm")
        farm_number = f"Farm {parts[-1].strip()}"
        full_name = parts[0].strip()

    # Split remaining full_name into potential first and last names
    name_parts = full_name.split()
    if len(name_parts) > 1:
        # Infer last_name and first_name from the reference sheet
        last_name, first_name = name_parts[-1], " ".join(name_parts[:-1])
        # Validate against the reference sheet
        match = name_reference[
            (name_reference['last_name'] == last_name) & 
            (name_reference['first_name'].str.contains(first_name, na=False))
        ]
        if not match.empty:
            # Use the first matched name for consistency
            first_name = match.iloc[0]['first_name']
            last_name = match.iloc[0]['last_name']
    else:
        # If only one part, consider it as last_name, first_name unknown
        last_name = name_parts[0]
        first_name = None

    return pd.Series([last_name, first_name, farm_number])

# Step 3: Apply the function to process the data
name_reference = sheet2[['last_name', 'first_name']]
sheet1[['last_name', 'first_name', 'farm_number']] = sheet1.apply(
    process_full_name, axis=1, name_reference=name_reference
)

# Display the updated data
print(sheet1.head())

# Save the updated sheet1 to a new Excel file
output_path = r"C:\Users\CHAKU FOODS\Documents\Updated Names and Farms.xlsx"
sheet1.to_excel(output_path, index=False)
